# Análise de Sentimento de Ações do Mercado Financeiro utilizando como chave o código do Ticker (Ex. PETR4 - Petrobras PN)

In [12]:
import configparser

config = configparser.ConfigParser()
config.read_file(open('../twitter.conf'))

CONSUMER_KEY = config.get('SECRETS', 'CONSUMER_KEY')
CONSUMER_SECRET = config.get('SECRETS', 'CONSUMER_SECRET')
ACCESS_TOKEN = config.get('SECRETS', 'ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = config.get('SECRETS', 'ACCESS_TOKEN_SECRET')

In [16]:
import nltk
nltk.download('vader_lexicon')

# nltk.sentiment é uma Biblioteca Python para análise de sentimento treinada em língua inglesa
#from nltk.sentiment import SentimentIntensityAnalyzer

# LeIA é uma Biblioteca Python para análise de sentimento treinada em língua portuguesa
from LeIA import SentimentIntensityAnalyzer
import pandas as pd
from bs4 import BeautifulSoup
import tweepy

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/marcelo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Lista de sites de mídia especializada em Mercado Financeiro no formato RSS e/ou Feed

In [15]:
symbols = pd.read_csv('data/tickers_list_to_analisys.csv')
symbols['symbol'] = symbols['symbol'].str.lower()
symbols['symbol'] = symbols['symbol'].str.removesuffix('.sa')
symbols['symbol'] = '#' + symbols['symbol']

symbols

,symbol
0,#rrrp3
1,#alpa4
2,#abev3
3,#amer3
4,#arzz3
...,...
84,#vale3
85,#viia3
86,#vbbr3
87,#wege3


In [17]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

TypeError: API.__init__() got an unexpected keyword argument 'wait_on_rate_limit_notify'

In [25]:
sia = SentimentIntensityAnalyzer()

titulos = []
links = []
sentimentos = []
valores_sentimentos = []

for url in urls:
  feed = fp.parse(url)
  for entry in feed.entries:
    titulo = entry.title
    link = entry.link

    content = ''
    if entry.has_key('content'):      
      for html in entry['content']:
        try:
          soup = BeautifulSoup(html['value'], features='html.parser')
          content += soup.get_text() + ' - '          
        except Exception as e:
          print(e)
          continue
    conteudo = entry.description + ' - ' + content
    
    texto = titulo + ' ' + conteudo    
    valor_sentimento = sia.polarity_scores(texto)
    sentimento = 'positivo' if valor_sentimento['compound'] > 0.05 else 'negativo'

    titulos.append(titulo)
    links.append(link)
    sentimentos.append(sentimento)
    valores_sentimentos.append(valor_sentimento['compound'])
  

In [26]:
df = pd.DataFrame({'titulo': titulos, 'link': links, 'sentimento': sentimentos, 'valor_sentimento': valores_sentimentos})
df

,titulo,link,sentimento,valor_sentimento
0,Sem votar esses textos sempre há algum prejuíz...,https://www.infomoney.com.br/politica/sem-vota...,negativo,-0.7351
1,Reforma tributária: relator reduz imposto para...,https://www.infomoney.com.br/politica/reforma-...,positivo,0.9674
2,Venezuela aumentará preços do diesel e cortará...,https://www.infomoney.com.br/economia/venezuel...,positivo,0.5859
3,Operação padrão de servidores do BC adia relat...,https://www.infomoney.com.br/economia/operacao...,negativo,-0.1531
4,Lira: Não há possibilidade de adiar a votação ...,https://www.infomoney.com.br/politica/lira-nao...,positivo,0.9810
...,...,...,...,...
88,“Imposto do pecado”: como ficam as ações da Am...,https://einvestidor.estadao.com.br/mercado/imp...,positivo,0.9858
89,Como a reforma tributária pode afetar 16 setor...,https://einvestidor.estadao.com.br/mercado/ref...,positivo,0.9989
90,Abertura de Mercado: expectativa pelos dados d...,https://einvestidor.estadao.com.br/mercado/abe...,negativo,-0.4939
91,Estas são as 25 ações que mais pagaram dividen...,https://einvestidor.estadao.com.br/mercado/aco...,positivo,0.9881


In [28]:
symbols['media'] = 0
symbols['sentimento'] = 'neutro'

for s in symbols['symbol'].values:
    id_sentimento = df['titulo'].str.contains(s)
    if id_sentimento.sum() > 0:
      #print(s, '\n', df['valor_sentimento'][id_sentimento])
      media = df['valor_sentimento'][id_sentimento].mean()
      symbols.loc[symbols['symbol'] == s, 'media'] = media

symbols.loc[symbols['media'] > 0.05, 'sentimento'] = 'positivo'
symbols.loc[symbols['media'] < -0.05, 'sentimento'] = 'negativo'

# Análise de Sentimento de ações que contém publicação na mídia especializada

In [29]:
symbols[symbols['media'] != 0]

,symbol,media,sentimento
0,RRRP3,0.9869,positivo
58,CASH3,0.8862,positivo
63,PETR3,0.9869,positivo
64,PETR4,0.5946,positivo
87,WEGE3,0.9869,positivo
